In [ ]:
import albumentations as A 
import matplotlib.pyplot as plt
import numpy as np
import torch

from importlib import import_module
from skp.toolbox.functions import load_model_from_config, plot_3d_image

In [ ]:
cfg = import_module("skp.configs.totalclassifier.cfg_seg_2d_to_3d").cfg 
dataset = import_module(f"skp.datasets.{cfg.dataset}").Dataset(cfg, mode="train")
model = import_module(f"skp.models.{cfg.model}").Net(cfg)

In [3]:
# model = load_model_from_config(cfg, weights_path="/home/ian/projects/SKP/experiments/totalclassifier/totalclassifier.cfg_seg_cls_mpr/8f7d5df4/fold0/checkpoints/last.ckpt", device="cuda", eval_mode=True)

In [ ]:
print("dataset size", len(dataset))
idx = np.random.randint(len(dataset))
batch = dataset[idx]

In [ ]:
print(batch["x"].shape)

In [ ]:
plot_3d_image(batch["x"].numpy()[:, 0], axis=0, num_images=16)
plot_3d_image(batch["y"].numpy(), axis=0, num_images=16)

In [ ]:
print("dataset size", len(dataset))
idx = np.random.randint(len(dataset))
batch = dataset[idx]

x = batch["seg"]["x"]
print(x.shape)
y = batch["seg"]["y"]
print(y.shape)

with torch.inference_mode():
    out = model({"seg": {"x": x.unsqueeze(0).cuda()}})

logits = out["seg"]["logits"].sigmoid()
logits = torch.cat([1 - logits.amax(1, keepdim=True), logits], dim=1)
labels = logits.argmax(1)[0].cpu().numpy()

plt.subplot(1, 3, 1)
plt.imshow(x[0].numpy(), cmap="gray")
plt.subplot(1, 3, 2)
plt.imshow(y.numpy(), cmap="gray")
plt.subplot(1, 3, 3)
plt.imshow(labels, cmap="gray")
plt.show()

In [ ]:
model({"x": batch["x"].unsqueeze(0)})["logits"].shape

In [ ]:
batch["y"]

In [ ]:
from skp.toolbox.functions import count_parameters

count_parameters(model)
count_parameters(model.transformer_head)

In [ ]:
y = batch["y"].numpy()
print(y.sum((1, 2)))
y = y.transpose(1, 2, 0)[..., -3:]
y = (y * 255).astype("uint8")

plt.imshow((0.6 * np.repeat(np.expand_dims(x, axis=-1), 3, axis=-1) + (0.4 * y)).astype("uint8"), cmap="gray")
plt.show()

In [ ]:
x = batch["seg"]["y"].numpy().transpose(1, 2, 0)[..., :3]
print(np.unique(x))
plt.imshow(x, cmap="gray")
plt.show()

In [ ]:
batch["cls"]["y"]

In [ ]:
dataset.inputs[idx]